In [2]:
#Importing and Reading
import pandas as pd
from datetime import datetime
df = pd.read_excel("Data.xlsx")
df.head()

,Store,Year,Month,Super Division,Sales,SGM,Store Open,Store Close,Outlet Type,State,Climate,Total Sq Ft,Store Modification Date,Store Modification
0,1,2015,2,GIRLS,63635.80,35692.09,1991-03-01,No Close date,Outlet,A1,Frigid,4735.0,2011-10-06 00:00:00,relocation
1,1,2015,2,BOYS,20872.93,15691.34,1991-03-01,No Close date,Outlet,A1,Frigid,4735.0,2011-10-06 00:00:00,relocation
2,1,2015,2,ACCESSORIES,38717.03,23866.25,1991-03-01,No Close date,Outlet,A1,Frigid,4735.0,2011-10-06 00:00:00,relocation
3,1,2015,2,KIDS,11455.00,12110.00,1991-03-01,No Close date,Outlet,A1,Frigid,4735.0,2011-10-06 00:00:00,relocation
4,1,2015,3,GIRLS,67507.77,42237.79,1991-03-01,No Close date,Outlet,A1,Frigid,4735.0,2011-10-06 00:00:00,relocation


In [24]:
# Parsing date columns
df['Store Open'] = pd.to_datetime(df['Store Open'], errors='coerce')
df['Store Close'] = pd.to_datetime(df['Store Close'], errors='coerce')
df['Store Modification Date'] = pd.to_datetime(df['Store Modification Date'], errors='coerce')

Initial Exploration

In [41]:
# 1. Total sales by year
sales_by_year = df.groupby('Year')['Sales'].sum().reset_index(name='Total Sales')
print("1. Total Sales by Year:\n", sales_by_year)

1. Total Sales by Year:
    Year   Total Sales
0  2015  1.627585e+09
1  2016  1.843939e+09
2  2017  1.651322e+09
3  2018  1.492340e+09
4  2019  1.471963e+09
5  2020  6.510899e+07


In [40]:
# 2. Number of stores opened in the year 1991
stores_1991 = df[df['Store Open'].dt.year == 1991]['Store'].nunique()
print(f"2. Number of stores opened in 1991: {stores_1991}")

2. Number of stores opened in 1991: 4


In [33]:
# 3. Number of stores remodelled (any type)
remodelled_stores = df[df['Store Modification Date'].notna()]['Store'].nunique()
print(f"3. Number of stores remodelled: {remodelled_stores}")

3. Number of stores remodelled: 346



In [39]:
# 4. Correlation between Sales and Total Sq Ft
corr_sales_sqft = df['Sales'].corr(df['Total Sq Ft'])
print(f"4. Correlation between Sales and Total Sq Ft: {corr_sales_sqft:}")

4. Correlation between Sales and Total Sq Ft: 0.13922544019446748



In [8]:
# 5. Most profitable super division
profit_by_div = df.groupby('Super Division')['SGM'].sum().reset_index(name='Total SGM')
top_div = profit_by_div.loc[profit_by_div['Total SGM'].idxmax(), 'Super Division']
print(f"5. Most profitable Super Division: {top_div}")

5. Most profitable Super Division: GIRLS



In [47]:
# 6. Number of active stores as of today
active_stores = df[df['Store Close'].isna()]['Store'].nunique()
print(f"6. Number of active stores as of today: {active_stores}")

6. Number of active stores as of today: 678


In [46]:
# 7. Super division with highest average sq ft
avg_sqft_div = df.groupby('Super Division')['Total Sq Ft'].mean().reset_index(name='Avg Sq Ft')
top_sqft_div = avg_sqft_div.loc[avg_sqft_div['Avg Sq Ft'].idxmax(), 'Super Division']
print(f"7. Super Division with highest average sq ft: {top_sqft_div}")

7. Super Division with highest average sq ft: KIDS


Advanced Insights

In [42]:
# Top 3 states by average sales
state_sales = df.groupby('State')['Sales'].mean().reset_index(name='Avg Sales')
top_states = state_sales.sort_values('Avg Sales', ascending=False).head(3)
print("Top 3 States by Average Sales:\n", top_states)

Top 3 States by Average Sales:
    State     Avg Sales
43    E4  58183.380314
16    B6  50044.942593
15    B5  49776.118243


In [44]:
# Best month to open a store (highest average sales)
month_sales = df.groupby('Month')['Sales'].mean().reset_index(name='Avg Sales')
best_month = month_sales.loc[month_sales['Avg Sales'].idxmax(), 'Month']
print(f"Best month to open a store (by average sales): Month {best_month}")

Best month to open a store (by average sales): Month 12


In [45]:
# Outlet-type effect on closures
outlet_stats = df.groupby('Outlet Type').agg(
    total_stores=('Store', 'nunique'),
    closed_stores=('Store Close', lambda x: x.notna().sum())
).reset_index()
outlet_stats['Closure Rate'] = outlet_stats['closed_stores'] / outlet_stats['total_stores']
print("Outlet-Type Closure Rates:\n", outlet_stats)

Outlet-Type Closure Rates:
   Outlet Type  total_stores  closed_stores  Closure Rate
0        Mall           383           5122     13.373368
1      Outlet           110           4845     44.045455
2       Strip           337          13786     40.908012
